# Two-dimensional harmonic osci

In the following we set $\hbar = c = 1$.

In [91]:
import numpy as np
import pickle

## Two-body matrix elements

We begin by loading the two-body orbital matrix elements from `dat/coulomb.dat`. In this example the orbitals are equal for both spin-directions thus the spin-orbital matrix elements has twice the number of basis functions in each dimension. Labeling the orbital matrix elements by $(\tilde{p}\tilde{q}\lvert u \rvert \tilde{r}\tilde{s})$, we wish to set up the anti-symmetric matrix elements by

\begin{align}
    u^{pq}_{rs}
        &= \langle pq\lvert u\rvert rs\rangle
        - \langle pq\lvert u\rvert sr\rangle \\
        &= \delta_{\sigma_p\sigma_r}\delta_{\sigma_q\sigma_s}
        (\tilde{p}\tilde{q}\lvert u\rvert \tilde{r}\tilde{s})
        - \delta_{\sigma_p\sigma_s}\delta_{\sigma_q\sigma_r}
        (\tilde{p}\tilde{q}\lvert u\rvert \tilde{s}\tilde{r}),
\end{align}
where $p = (\tilde{p}, \sigma_p)$.

In [76]:
hbar = 1

In [77]:
l = 12 # Number of spin-orbitals
n = 2 # Number of particles
orbital_integrals = np.zeros((l//2, l//2, l//2, l//2))

with open("dat/coulomb.dat", "r") as f:
    for row in f.read().split("\n"):
        row = row.split()

        if row:
            p, q, r, s, val = row
            orbital_integrals[int(p), int(q), int(r), int(s)] = float(val)

Having read in the orbital matrix elements, i.e., $(\tilde{p}\tilde{q}\lvert u\rvert \tilde{r}\tilde{s})$, we construct the anti-symmetric elements, $u^{pq}_{rs}$, using the formula shown above. We therefore introduce two $\delta$-functions. One for the spatial part and one for the spin part.

In [78]:
# Regular Kronecker-delta for indices
delta = lambda p, q: p == q
# Spin-delta, checking if both indices are odd or even
spin_delta = lambda p, q: not ((p & 0x1) ^ (q & 0x1))

In [79]:
u = np.zeros((l, l, l, l))

for p in range(l):
    for q in range(l):
        for r in range(l):
            for s in range(l):
                pqrs = spin_delta(p, r) * spin_delta(q, s) * orbital_integrals[p//2, q//2, r//2, s//2]
                pqsr = spin_delta(p, s) * spin_delta(q, r) * orbital_integrals[p//2, q//2, s//2, r//2]
                u[p, q, r, s] = pqrs - pqsr

Below we test if the anti-symmetric matrix elements are actually anti-symmetric, i.e.,

\begin{align}
    u_{rs}^{pq} = -u_{sr}^{pq} = -u_{rs}^{qp} = u_{sr}^{qp}.
\end{align}

In [86]:
tol = 1e-7

for p in range(l):
    for q in range(l):
        for r in range(l):
            for s in range(l):
                assert (abs(u[p, q, r, s] - u[q, p, s, r]) < tol)
                assert (abs(u[p, q, r, s] + u[q, p, r, s]) < tol)
                assert (abs(u[p, q, r, s] + u[p, q, s, r]) < tol)

## One-body matrix elements

We look at the _two-dimensional harmonic oscillator quantum dot_ (HOQD 2D) with energy levels given by

\begin{align}
    \epsilon_{n, m} = \hbar\omega(2n + |m| + 1).
\end{align}

See `books/articles/QDots.pdf` page 15 for the derivation of this expression.

In [87]:
# To use this function, we must provide a mapping from (p, q) -> (n, m)
epsilon = lambda n, m, omega=1: hbar*omega*(2*n + abs(m) + 1)

In [82]:
h = np.zeros((l, l))
omega = 1

for p in range(l):
    if p < 2:
        h[p, p] = omega
    elif 2 <= p < 6:
        h[p, p] = 2*omega
    elif 6 <= p < 12:
        h[p, p] = 3*omega

## Calculating reference energy

The coupled-cluster doubles energy can be calculated by

\begin{align}
    E_{\text{CCD}} = h_i^i + \frac{1}{2}u_{ij}^{ij}
        + \frac{1}{4}u_{ij}^{ab}t_{ij}^{ab},
\end{align}
where $t$ are the cluster amplitudes (to be determined later). The first part is a reference energy we use for testing the current values of $h$ and $u$.

\begin{align}
    E_{\text{ref}} = h_i^i + \frac{1}{2}u_{ij}^{ij}.
\end{align}

For 2D HOQD with $n = 2$ and $l = 12$, we should get $E_{\text{ref}} \approx 3.25 \text{Hartree}$.

In [89]:
e_ref = np.einsum("ii->", h[:n, :n]) + 0.5*np.einsum("ijij->", u[:n, :n, :n, :n])
print ("E_ref = {0:.4f} Hartree".format(e_ref))

E_ref = 3.2533 Hartree


## Pickling the result to be used in the CCD-notebook

In [101]:
pickle.dump(h, open("dat/h.pkl", "wb"))
pickle.dump(u, open("dat/u.pkl", "wb"))